# In this notebook, we would like to solve imdb review sentiment classification task via an RNN approach. More specifically by an LSTM

# import packages


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import pandas as pd
import os
import gc
import numpy as np

import os
import sys
import pandas as pd
import numpy as np
seed= 7
np.seed= seed
import gc
import importlib
import pickle
from pprint import pprint
import time


# visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set(style="whitegrid", font_scale= 1.5)
from IPython.display import display

# modelling
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# text preprocessing
from nltk.corpus import stopwords
import string


# Neural Networks
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer


"""from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate"""
print()


Using TensorFlow backend.


# import data

In [2]:
data= pd.read_csv( '/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
print(data.shape)
data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
print( data['review'].iloc[3] )

Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.


# train_test_split

In [4]:
data['sentiment'].value_counts(normalize= True)

positive    0.5
negative    0.5
Name: sentiment, dtype: float64

In [5]:
data_train, data_test= train_test_split (data, test_size=0.2, random_state= seed, 
                                        shuffle= True, stratify= data['sentiment'])

X_train= data_train['review']; X_train.index= range(len(X_train))
y_train= data_train['sentiment']; y_train.index= range(len(y_train))
X_test= data_test['review']; X_test.index= range(len(X_test))
y_test= data_test['sentiment']; y_test.index= range(len(y_test))

print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print()
print( 'distribution of sentiments in the train data: \n', 
              data_train['sentiment'].value_counts(normalize= True) )
print()
print( 'distribution of sentiments in the test data: \n',  
              data_test['sentiment'].value_counts(normalize= True) )

del(data_train, data_test); gc.collect()

X_train shape:  (40000,)
X_test shape:  (10000,)

distribution of sentiments in the train data: 
 positive    0.5
negative    0.5
Name: sentiment, dtype: float64

distribution of sentiments in the test data: 
 positive    0.5
negative    0.5
Name: sentiment, dtype: float64


22

# preprocessing the text

In [6]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

X_train= X_train.apply(lambda row: clean_doc(row) )
X_test= X_test.apply(lambda row: clean_doc(row) )

y_train= y_train.apply( lambda x: 1 if x=='positive' else 0)
y_test= y_test.apply( lambda x: 1 if x=='positive' else 0)

In [7]:
display(X_train.head(2))

0    [There, good, moments, film, Only, couple, cha...
1    [Soylent, Green, classic, waiting, someone, re...
Name: review, dtype: object

**Why do we tokenize text data?**

Most NLP applications need an embedding layer. An embedding layer is usually put in the first layer, and its purpose is to learn meaningful representations of words. There are a variety of neural network architechture to learn these embeggings. The input of an embedding layer is a fixed-sized window of words, and the output is their embeddings. The mechanism is explained by me in one sheet of paper. 

However, the layer cannot accept words as they are. In reality, we need to one-hot encode the words first. However, an equivalent way of one-hot encoding is integer encoding of words. Every word in the vocabulary gets a unique integer assigned to it. This process is sometimes called tokenization. The following function implements it.

In [8]:
# fit a tokenizer
def create_tokenizer(lines):
    """
    returns an object for to tokenize words in each document of our dataset.
    lines is a list of lists. Each sublist in lines (representing one document) 
    is a list of words.
    
    Ex. 
    
    integer_encoded= create_tokenizer(lines).tests_to_sequences( texts )
    texts is a dataframe or series or list of elments where elements are lists of words
    
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

We need to find out the maximum length of our documents. If you remember, our documents are now in form of a list of words. We need to know the maximum length because later on we'll have to pad shorter documents with zeros and by as much as this maximum size.

In [9]:
# calculate the maximum document length
def max_length(lists):
    """
    lists is a list of lists
    It returns the maximum length of its elements
    """
    return max([len( l ) for l in lists])

max( max_length( X_train.tolist() ), max_length( X_test.tolist() ) )

1480

In [10]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    """
    Arguments:
        tokenizer is a tokenizer object, like the one we fit before
        lines is a list of elements where each element is a list of words, and it stands for one document
        length is the maximum length of documents in the train and test data
    
    It integer encodes the texts, pad them with zeros by as much as length and returns a list of lists.
    """    
    
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = sequence.pad_sequences(encoded, maxlen= length )
    return padded

tokenizer= create_tokenizer( X_train.tolist() )
length= max( max_length( X_train.tolist() ), max_length( X_test.tolist() ) )
encoded_train= encode_text( tokenizer, X_train.tolist(), length )
encoded_test= encode_text( tokenizer, X_test.tolist(), length )

# model architechture

In [11]:
embedding_vector_length= 100
# we add 1 because of 0 from padding
vocab_size = len(tokenizer.word_index) + 1
# length is the maximum review length 
length= max( max_length( X_train.tolist() ), max_length( X_test.tolist() ) )


model= Sequential()
model.add( Embedding(vocab_size, embedding_vector_length, input_length= length ))
model.add( LSTM(100) )
model.add( Dense(1, activation= 'sigmoid') )
model.compile( loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
model.summary()

model.fit(encoded_train, y_train, epochs= 3, batch_size= 1024)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1480, 100)         15462100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 15,542,601
Trainable params: 15,542,601
Non-trainable params: 0
_________________________________________________________________


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
40000/40000 [==============================] - 69s 2ms/step - loss: 0.6212 - accuracy: 0.6777
Epoch 2/3
40000/40000 [==============================] - 68s 2ms/step - loss: 0.4892 - accuracy: 0.8206
Epoch 3/3
40000/40000 [==============================] - 67s 2ms/step - loss: 0.2507 - accuracy: 0.8996


In [12]:
scores= model.evaluate( encoded_test, y_test)
print('Accuracy: %.2f %%' % (scores[1]*100))

10000/10000 [==============================] - 67s 7ms/step
Accuracy: 87.45 %
